In [30]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import numpy as np
import sys
import warnings
import numpy as np
from scipy.signal import filtfilt
from scipy.signal import firwin
from scipy.signal import resample
warnings.filterwarnings('ignore')

DATA_PATH = '/workspaces/Tracking-Gazes-on-Museum-Pieces-Data-Plus/data'

def concat_df(DATA_PATH):
    df = pd.DataFrame()
    for folder in os.listdir(DATA_PATH):
        if not os.path.isdir(os.path.join(DATA_PATH, folder)):
            continue
        temp_path = os.path.join(DATA_PATH, folder, 'gaze.csv')
        temp_df = pd.read_csv(temp_path)
        temp_df['folder_id'] = folder
        df = pd.concat([df, temp_df])
    return df


demographic_data = pd.read_excel(os.path.join(DATA_PATH, 'demographic.xlsx'))
orig_data = concat_df(DATA_PATH)
modified_data = pd.read_csv(os.path.join(DATA_PATH, 'all_gaze.csv'), compression='gzip')

# keep_columns = ['timestamp [ns]_for_grouping', 'participant_folder', 'ref_center_x', 'ref_center_y']
# modified_data = modified_data[keep_columns]
eyedat = []
for idx, folder in enumerate(orig_data['folder_id'].unique()):
    temp = orig_data[orig_data['folder_id'] == folder]
    eyedat.append(temp[['gaze x [px]', 'gaze y [px]']].to_numpy())


In [33]:
### Supporting functions

"""
Explanation:

The Python function BehavioralIndex takes behavind as an input parameter, representing the behavioral indexes.
The line dind = diff(behavind) in MATLAB calculates the differences between consecutive elements in behavind. In Python, this is done using np.diff(behavind).
The line gaps = find(dind > 1) in MATLAB finds the indices where the differences in dind are greater than 1. In Python, this is achieved with np.where(dind > 1)[0].
The line behaveind = zeros(length(gaps),50) in MATLAB initializes a 2D array behaveind with zeros, where the number of rows is determined by the length of gaps and the number of columns is fixed at 50. In Python, this is equivalent to np.zeros((len(gaps), 50)).
The subsequent if statement checks if there are any gaps found (if ~isempty(gaps) in MATLAB). If there are gaps, the for loop is executed to fill the behaveind array accordingly. Otherwise, behaveind is assigned the value of behavind.
Inside the for loop, the current gap index (gapind) is used to extract the corresponding elements from behavind and assign them to temp in both MATLAB and Python.
The line behaveind(gapind,1:length(temp)) = temp in MATLAB assigns the values from temp to the appropriate row in behaveind. In Python, this is done using behaveind[gapind, :len(temp)] = temp.
The for loop ends, and the control reaches the else block, which assigns behaveind to behavind if there are no gaps found (behaveind = behavind in MATLAB and Python).
The line behaviortime = zeros(2,size(behaveind,1)) in MATLAB initializes a 2D array behaviortime with zeros, where the number of rows is fixed at 2 and the number of columns is determined by the size of behaveind. In Python, this is equivalent to np.zeros((2, behaveind.shape[0])).
The subsequent for loop iterates over the range from 1 to the number of rows in behaveind, using range(behaveind.shape[0]) in Python.
Inside the loop, the line rowfixind = behaveind(index,:) in MATLAB extracts the current row of behaveind. In Python, this is achieved using rowfixind = behaveind[index, :].
The line rowfixind(rowfixind == 0) = [] in MATLAB removes any zero elements from rowfixind. In Python, this can be done using rowfixind = rowfixind[rowfixind != 0].
The line behaviortime(:,index) = [rowfixind(1);rowfixind(end)] in MATLAB assigns the first and last elements of rowfixind to the appropriate column in behaviortime. In Python, this is done using behaviortime[:, index] = [rowfixind[0], rowfixind[-1]].
The for loop ends, and the behaviortime array is returned as the output of the function.
"""
def BehavioralIndex(behavind):
    # Function turns indexes into times by parsing at breaks in continuity
    dind = np.diff(behavind)
    gaps = np.where(dind > 1)[0]
    behaveind = np.zeros((len(gaps), 50))
    
    if len(gaps) > 0:
        for gapind in range(len(gaps)):
            if gapind == 0:
                temp = behavind[:gaps[gapind]]
            elif gapind == len(gaps):
                temp = behavind[gaps[gapind - 1]:]
            else:
                
                temp = behavind[gaps[gapind - 1]:gaps[gapind]]

            behaveind[gapind, :len(temp)] = temp
    else:
        behaveind = behavind
    
    behaviortime = np.zeros((2, behaveind.shape[0]))
    
    for index in range(behaveind.shape[0]):
        rowfixind = behaveind[index, :]
        rowfixind = rowfixind[rowfixind != 0]
        behaviortime[:, index] = [rowfixind[0], rowfixind[-1]]
    
    return behaviortime
"""
Explanation:
The code is similar to the previous BehavioralIndex function, with the addition of calculating the mean fixation position.

The function BehavioralIndexXY takes three input parameters: behavind (behavioral indexes), x (x-coordinate data), and y (y-coordinate data).
The code starts by calculating the differences between consecutive elements of behavind using np.diff(behavind).
The line gaps = np.where(dind > 1)[0] finds the indices where the differences in dind are greater than 1.
The subsequent code is similar to the previous function, where behaveind is populated based on the gaps found and assigned the values of behavind if no gaps are found.
Arrays behaviortime and behaviormean are initialized with appropriate sizes.
The loop iterates over the rows of behaveind and performs similar operations as before, calculating the fixation times and mean fixation positions.
The calculated values are assigned to the respective columns in behaviortime and behaviormean.
The function returns both behaviortime and behaviormean as the output.
Note: The code assumes the necessary imports have been done, such as import numpy as np to use NumPy functions and arrays.

"""
def BehavioralIndexXY(behavind, x, y):
    # Function is the same as above but also calculates mean fixation position
    dind = np.diff(behavind)
    gaps = np.where(dind > 1)[0]
    behaveind = np.zeros((len(gaps), 50))
    
    if len(gaps) > 0:
        for gapind in range(len(gaps) + 1):
            if gapind == 0:
                temp = behavind[:gaps[gapind]]
            elif gapind == len(gaps) + 1:
                temp = behavind[gaps[gapind - 1] + 1:]
            else:
                temp = behavind[gaps[gapind - 1] + 1:gaps[gapind]]
            behaveind[gapind, :len(temp)] = temp
    else:
        behaveind = behavind
    
    behaviortime = np.zeros((2, behaveind.shape[0]))
    behaviormean = np.zeros((2, behaveind.shape[0]))
    
    for index in range(behaveind.shape[0]):
        rowfixind = behaveind[index, :]
        rowfixind = rowfixind[rowfixind != 0]
        behaviortime[:, index] = [rowfixind[0], rowfixind[-1]]
        behaviormean[:, index] = [np.mean(x[rowfixind]), np.mean(y[rowfixind])]
    
    return behaviortime, behaviormean


"""
Explanation:
The code extracts variables from fixation or saccades based on the input data xss, yss, and samprate.

The function extractVariables takes three input parameters: xss (x-coordinate data), yss (y-coordinate data), and samprate (sampling rate).
The code checks if the length of xss is greater than or equal to 3.
If the condition is satisfied, the code proceeds to calculate the velocity, angle, and acceleration based on the provided formulas.
An array pp of size 6 is initialized.
The maximum velocity and acceleration are assigned to pp[0] and pp[1], respectively.
Arrays dist and rot are initialized with appropriate sizes.
The subsequent for loop iterates over the range from 0 to the length of xss minus 2.
Inside the loop, the code calculates the rotation and distance based on the given formulas.
The line rot[rot > 180] = rot[rot > 180] - 180 adjusts the values of rot to be within the range of 0 to 180.
The mean distance, mean velocity, absolute mean angle, and mean rotation are assigned to pp[2], pp[3], pp[4], and pp[5], respectively.
If the length of xss is less than 3, the code assigns pp as an array of NaN values.
The function returns pp as the output.
"""

def extractVariables(xss, yss, samprate):
    # Code below extracts variables from fixation or saccades
    if len(xss) >= 3:
        vel = np.sqrt(np.diff(xss)**2 + np.diff(yss)**2) / samprate
        angle = 180 / np.pi * np.arctan2(np.diff(yss), np.diff(xss))
        accel = np.abs(np.diff(vel)) / samprate
        pp = np.zeros(6)
        pp[0] = np.max(vel)
        pp[1] = np.max(accel)
        dist = np.zeros(len(xss) - 2)
        rot = np.zeros(len(xss) - 2)
        for aa in range(len(xss) - 2):
            rot[aa] = np.abs(angle[aa] - angle[aa + 1])
            dist[aa] = np.sqrt((xss[aa] - xss[aa + 2])**2 + (yss[aa] - yss[aa + 2])**2)
        rot[rot > 180] = rot[rot > 180] - 180
        pp[2] = np.mean(dist)
        pp[3] = np.mean(vel)
        pp[4] = np.abs(np.mean(angle))
        pp[5] = np.mean(rot)
    else:
        pp = np.full(6, np.nan)
    return pp


"""
Explanation:
The code calculates the inter vs intra distance points by cluster using a modified version of the silhouette method.

The function InterVSIntraDist takes two input parameters: X (data points) and clust (cluster assignments).
The code first converts the cluster assignments clust into unique indices using the np.unique function. The unique indices are stored in the variable idx, and the inverse mapping of clust to indices is stored in cnames.
The variables n and k are assigned the lengths of idx and cnames, respectively.
The code uses np.histogram to count the occurrences of each cluster, resulting in the count array.
The variable mbrs is created as a binary matrix indicating the membership of each point to each cluster.
The code initializes arrays avgDWithin and avgDBetween with values of infinity.
A nested loop is used to calculate the average distances within and between clusters.
The outer loop iterates over the range of n, representing each data point.
The variable distj calculates the squared distances between the current data point and all other data points.
The inner loop iterates over the range of k, representing each cluster.
If the cluster index matches the index of the current data point, the code calculates the average distance within the cluster by summing the distances weighted by membership and dividing by the count minus 1.
Otherwise, the code calculates the average distance between the current data point and the other cluster by summing the distances weighted by membership and dividing by the count.
The code calculates the minimum average distance between clusters for each data point and assigns it to minavgDBetween.
The silhouette values are calculated using the formula: (minavgDBetween - avgDWithin) / max(avgDWithin, minavgDBetween).
The calculated silhouette values are assigned to silh and returned as the output of the function.
Note: The code assumes the necessary imports have been done, such as import numpy as np to use NumPy functions and arrays.
"""
def InterVSIntraDist(X, clust):
    # Inter vs Intra distance points by cluster-mod of SILHOUETTE
    # by Seth Koenig October 21, 2012
    
    idx, cnames = np.unique(clust, return_inverse=True)
    n = len(idx)
    k = len(cnames)
    count = np.histogram(clust, bins=np.arange(k+1))[0]
    mbrs = np.equal.outer(cnames, np.arange(k)).astype(int)
    
    # Get avg distance from every point to all (other) points in each cluster
    myinf = np.inf
    avgDWithin = np.full(n, myinf)
    avgDBetween = np.full((n, k), myinf)
    for j in range(n):
        distj = np.sum((X - X[j])**2, axis=1)
        # Compute average distance by cluster number
        for i in range(k):
            if i == cnames[j]:
                avgDWithin[j] = np.sum(distj * mbrs[:, i]) / max(count[i] - 1, 1)
            else:
                avgDBetween[j, i] = np.sum(distj * mbrs[:, i]) / count[i]
    
    # Calculate the silhouette values
    minavgDBetween = np.min(avgDBetween, axis=1)
    silh = (minavgDBetween - avgDWithin) / np.maximum(avgDWithin, minavgDBetween)
    
    return silh


In [44]:
"""
Explanation:

1. The warnings module is imported to handle warning messages.
2. The warnings.filterwarnings() function is used to suppress specific warning messages. The category parameter specifies the type of warning to be ignored, and the module parameter specifies the module from which the warning is raised.
3. The if statement checks if the length of the command-line arguments (sys.argv) is less than 2. If so, it raises a ValueError with the message 'No data file found'.
4. The next if statement checks if the length of the command-line arguments is exactly 2. If so, it assigns the value 5 / 1000 to the samprate variable. This represents the sampling rate in seconds (e.g., 200 Hz -> 5 ms/sample -> 5/1000).
5. The variables list is initialized with the parameter names to be extracted.
6. The fltord, lowpasfrq, and nyqfrq variables are set to their respective values.
7. The signal.firwin2() function from the scipy.signal module is used to create a 30 Hz low pass filter. It takes the filter order (fltord), the frequency range ([0, lowpasfrq / nyqfrq, lowpasfrq / nyqfrq, 1]), and the corresponding amplitude values ([1, 1, 0, 0]) as parameters. The resulting filter coefficients are assigned to the flt variable.

8. The variable buffer is calculated as 100 / samprate / 1000, representing a 100 ms buffer for filtering.
9. An empty list fixationstats is created with a length equal to the number of elements in eyedat.
10. A loop iterates over the indices of eyedat using the range() function.
11. An if statement checks if the number of columns (eyedat[cndlop].shape[1]) in the current eyedat element is greater than 500 / samprate / 1000. If true, the subsequent code is executed.
12. Inside the filtering section, the variable x is assigned with the first row of eyedat[cndlop], representing the x-coordinate data.
13. The variable y is assigned with the second row of eyedat[cndlop], representing the y-coordinate data.
14. The np.concatenate() function is used to append the reversed x array with the original x array and then with the reversed x array again to create a buffer for filtering.
15. The same operation is performed for the y array.
16. The resample() function from scipy.signal is used to upsample the x and y arrays to a frequency of samprate * 1000 Hz.
17. The filtfilt() function applies the flt filter coefficients to the upsampled x and y arrays in a forward and backward direction, resulting in xss and yss respectively.
18. The xss and yss arrays are trimmed by removing the first 100 and last 100 elements, which correspond to the filter buffer, resulting in the filtered eye traces.
19. The lines x = x(101:end-100) and y = y(101:end-100) in MATLAB remove the filter buffer from the x and y arrays. In Python, the equivalent indexing is x[100:-100] and y[100:-100], respectively.
20. The lines velx = diff(xss) and vely = diff(yss) calculate the differences between consecutive elements of xss and yss arrays, respectively. In Python, np.diff() function is used for this purpose.
21. The line vel = sqrt(velx.^2+vely.^2) calculates the magnitude of the velocity vector using the Euclidean distance formula. In Python, this is achieved by np.sqrt(velx**2 + vely**2).
22. The line accel = abs(diff(vel)) calculates the absolute difference between consecutive elements of the vel array, representing acceleration. In Python, np.diff() and np.abs() functions are used to achieve the same result.
23. The line angle = 180*atan2(vely,velx)/pi calculates the angle of the velocity vector using the atan2() function and converts it from radians to degrees. In Python, np.degrees(np.arctan2(vely, velx)) is used for the same purpose.
24. The line vel = vel(1:end-1) removes the last element from the vel array. In Python, the equivalent is vel = vel[:-1].
25. The lines rot = zeros(1,length(xss)-2) and dist = zeros(1,length(xss)-2) initialize arrays rot and dist with zeros of length length(xss)-2. In Python, np.zeros() is used for the same purpose.
26. The subsequent for loop calculates the angular velocity and distance values. The loop iterates over the range of len(xss)-2 and assigns the calculated values to the corresponding indices of rot and dist arrays. The equivalent Python code utilizes the range() function and indexing syntax with square brackets.
27. The line rot(rot > 180) = rot(rot > 180)-180 in MATLAB adjusts the values in the rot array that are greater than 180 by subtracting 180. In Python, this operation is achieved with rot[rot > 180] = rot[rot > 180] - 180.
28. The line rot = 360-rot subtracts each element of the rot array from 360. This operation aims to make the angular velocity values small so that fixation values are also small.
29. The line points = [dist' vel' accel' rot'] combines the dist, vel, accel, and rot arrays column-wise to create a new array points. In Python, the np.column_stack() function is used for the same purpose.
30. The subsequent for loop iterates over the range of the number of columns in the points array, using points.shape[1].
31. Inside the loop, the variable thresh is calculated as the mean of the current column plus three times the standard deviation of the current column of the points array.
32. The line points((points(:,ii) > thresh),ii) = thresh in MATLAB replaces values in the points array that are greater than thresh with thresh. In Python, this is achieved by points[points[:, ii] > thresh, ii] = thresh.
33. The subsequent lines in the loop normalize the values in each column of the points array to the range [0, 1]:
34. points[:, ii] = points[:, ii] - np.min(points[:, ii]) subtracts the minimum value of the current column from each element of that column.
35. points[:, ii] = points[:, ii] / np.max(points[:, ii]) divides each element of the current column by the maximum value of that column.
36. The line sil = zeros(1,5) in MATLAB initializes an array sil of size 1x5 with zeros. In Python, the equivalent is sil = np.zeros(5).
37. The subsequent for loop iterates over the range from 2 to 5, using range(2, 6) in Python.
38. Inside the loop, the line T = kmeans(points(1:10:end,2:4),numclusts,'replicate',5) in MATLAB performs k-means clustering on a subset of the points array, using only columns 2 to 4 (velocity, acceleration, and angular velocity) and every 10th row. In Python, this is achieved using the KMeans class from the sklearn.cluster module. The fit_predict() method is used to assign cluster labels to the subset of points.
39. The line [silh] = InterVSIntraDist(points(1:10:end,2:4),T) in MATLAB calculates the silhouette values using the InterVSIntraDist function. In Python, the equivalent function is assumed to be defined separately and is referred to as InterVSIntraDist().
40. The line sil(numclusts) = mean(silh) in MATLAB assigns the mean value of silh to the sil array at the index corresponding to the current number of clusters. In Python, this is accomplished with sil[numclusts] = np.mean(silh).
41. The line sil(sil > 0.9*max(sil)) = 1 in MATLAB sets the elements in sil greater than 0.9 times the maximum value of sil to 1. In Python, this is achieved with sil[sil > 0.9 * np.max(sil)] = 1.
42. The line numclusters = find(sil == max(sil)) in MATLAB finds the index of the maximum value in sil. In Python, this can be achieved with numclusters = np.argmax(sil).
43. The line T = kmeans(points,numclusters(end),'replicate',5) in MATLAB performs k-means clustering on the points array using the determined number of clusters. In Python, this is done with T = KMeans(n_clusters=numclusters, n_init=5).fit_predict(points).
44. The lines meanvalues = zeros(max(T),size(points,2)) and stdvalues = zeros(max(T),size(points,2)) in MATLAB initialize the meanvalues and stdvalues arrays with zeros, with sizes determined by the maximum value in T and the number of columns in points. In Python, the equivalent is meanvalues = np.zeros((np.max(T), points.shape[1])) and stdvalues = np.zeros((np.max(T), points.shape[1])).
45. The subsequent for loop iterates over the range from 1 to the maximum value in T, using range(1, np.max(T) + 1) in Python.
46. Inside the loop, the line tc = find(T == TIND) in MATLAB finds the indices where T is equal to the current TIND value. In Python, this is achieved with tc = np.where(T == TIND)[0].
47. The lines meanvalues(TIND,:) = mean(points(tc,:)) and stdvalues(TIND,:) = std(points(tc,:)) in MATLAB calculate the mean and standard deviation of the points in tc along each column and assign them to the corresponding rows of meanvalues and stdvalues. In Python, this is done with meanvalues[TIND - 1, :] = np.mean(points[tc, :], axis=0) and stdvalues[TIND - 1, :] = np.std(points[tc, :], axis=0). Note the use of TIND - 1 to account for Python's 0-based indexing.
48. The variable fixationcluster is assigned the index of the cluster with the minimum sum of mean velocity and mean acceleration values.
49. All elements in T that have the value of fixationcluster are set to 100.
50. The variable fixationcluster2 is assigned the indices of clusters whose mean velocity is less than the mean velocity of fixationcluster plus 3 times the standard deviation of fixationcluster's velocity.
51. If any element in fixationcluster2 matches fixationcluster, it is removed.
52. A loop iterates over each index in fixationcluster2, and for each index, all elements in T that have the value of that index are set to 100.
53. All elements in T that are not equal to 100 are set to 2, indicating non-fixation clusters.
54. All elements in T that are equal to 100 are set to 1, indicating fixation clusters.

55. The variable fixationindexes is assigned the indices of elements in T that have the value of 1, indicating fixations.
60. The function BehavioralIndex is called to obtain fixationtimes by parsing the fixationindexes at breaks in continuity.
61. The code removes fixation durations shorter than 25 ms from fixationtimes using the condition np.diff(fixationtimes, axis=0) >= 25.
62. A loop iterates over each column in fixationtimes.
63. For each iteration, the indices altind are selected by taking a range around the fixation time points, with a 50 ms buffer on each side.
64. The indices in altind that are less than 0 or greater than or equal to the length of points are removed.
65. The submatrix POINTS is extracted from points using the indices in altind.
66. The code initializes an array sil to store silhouette values for different numbers of clusters.
67. Another loop iterates over the range from 1 to 5 (inclusive) to perform the local re-clustering.
68. For each iteration, the KMeans algorithm is used to cluster the POINTS data, with the number of clusters specified by numclusts.
69. The InterVSIntraDist function is called to compute silhouette values using the clustered points.
70. The mean silhouette value is computed and stored in the sil array.
71. Silhouette values that are greater than 0.9 times the maximum silhouette value are set to 1.
72. The number of clusters with the maximum silhouette value is obtained.
73. The KMeans algorithm is applied again to cluster the POINTS data, this time using the number of clusters specified by the rounded median of numclusters.
74. The resulting cluster labels are stored in T.
75. The array rng is initialized to store the ranges for each cluster.
76. The variable medianvalues is initialized as a zero matrix with dimensions based on the maximum cluster label in T and the number of columns in POINTS.
77. A loop iterates over each cluster label, from 1 to the maximum cluster label.
78. For each iteration, the indices tc are obtained by finding the elements in T.labels_ that match the current cluster label.
80. If the length of tc is 1, indicating a single point in the cluster, the range rng for that cluster is set to a vector of ones.
81. The median values for that cluster are assigned from the corresponding point in POINTS.
82. If the length of tc is greater than 1, indicating multiple points in the cluster, the maximum and minimum values of POINTS (excluding the last column) within the cluster are assigned to the range rng.
83. The median values for that cluster are computed by taking the median of POINTS within the cluster along each column.
84. The range rng and median values medianvalues are updated accordingly.
"""

import warnings
import scipy.signal as signal

warnings.filterwarnings("ignore", category=UserWarning, module="stats")  # Suppress unnecessary warning message
warnings.filterwarnings("ignore", category=UserWarning, module="stats")  # Suppress unnecessary warning message

samprate = 2000 / 1000  # in seconds, e.g. 200 Hz -> 5 ms/sample -> 5/1000

variables = ['Dist', 'Vel', 'Accel', 'Angular Velocity']  # Parameters to be extracted

fltord = 60
lowpasfrq = 30
nyqfrq = 1000 / 2
flt = signal.firwin2(fltord, [0, lowpasfrq / nyqfrq, lowpasfrq / nyqfrq, 1], [1, 1, 0, 0])  # 30 Hz low pass filter

buffer = int(100 / samprate / 1000)  # 100 ms buffer for filtering

fixationstats = [None] * len(eyedat)
for cndlop in range(len(eyedat)):
    if True: #eyedat[cndlop].shape[1] > 500 / samprate / 1000:
        # ---Filtering Extract Parameters from Eye Traces---%
        x = eyedat[cndlop][:, 0]  # * 24 + 400  # converts dva to pixel and data from [-400,400] to [0,800]
        y = eyedat[cndlop][:, 1]  # * 24 + 300  # converts dva to pixel and from [-300,300] to [0,600]

        x = np.concatenate((x[buffer::-1], x, x[-buffer - 1:-1:-1]))  # add buffer for filtering
        y = np.concatenate((y[buffer::-1], y, y[-buffer - 1:-1:-1]))  # add buffer for filtering
        x = resample(x, int(samprate * 1000))  # upsample to 1000 Hz
        y = resample(y, int(samprate * 1000))  # upsample to 1000 Hz

        xss = filtfilt(flt, 1, x)
        yss = filtfilt(flt, 1, y)
        xss = xss[100:-100]  # remove buffer after filtering
        yss = yss[100:-100]  # remove buffer after filtering
        x = x[100:-100]  # remove buffer after filtering
        y = y[100:-100]  # remove buffer after filtering

        velx = np.diff(xss)
        vely = np.diff(yss)
        vel = np.sqrt(velx**2 + vely**2)  # velocity
        accel = np.abs(np.diff(vel))  # acceleration
        angle = np.degrees(np.arctan2(vely, velx))
        vel = vel[:-1]
        rot = np.zeros(len(xss) - 2)  # angular velocity
        dist = np.zeros(len(xss) - 2)  # distance

        for a in range(len(xss) - 2):
            rot[a] = np.abs(angle[a] - angle[a+1])
            dist[a] = np.sqrt((xss[a] - xss[a+2])**2 + (yss[a] - yss[a+2])**2)
        rot[rot > 180] = rot[rot > 180] - 180
        rot = 360 - rot  # want angular velocity to be small so fixation values are all small

        points = np.column_stack((dist, vel, accel, rot))
        for ii in range(points.shape[1]):
            thresh = np.mean(points[:, ii]) + 3 * np.std(points[:, ii])
            points[points[:, ii] > thresh, ii] = thresh
            points[:, ii] = points[:, ii] - np.min(points[:, ii])
            points[:, ii] = points[:, ii] / np.max(points[:, ii])

        sil = np.zeros(5)  # determines the number of clusters by comparing the ratio of intercluster and intracluster distances, faster mod of silhouette
        for idx, numclusts in enumerate(range(2, 6)):
            # can save a little bit of computation by only using 3/4 parameters (-distance) and 1/10 of data points
            T = KMeans(n_clusters=numclusts, n_init=5).fit_predict(points[::10, 1:4])
            silh = InterVSIntraDist(points[::10, 1:4], T)
            sil[idx] = np.mean(idx)

        sil[sil > 0.9 * np.max(sil)] = 1
        numclusters = np.argmax(sil)
        T = KMeans(n_clusters=numclusters, n_init=5).fit_predict(points)
        meanvalues = np.zeros((np.max(T), points.shape[1]))
        stdvalues = np.zeros((np.max(T), points.shape[1]))

        for TIND in range(np.max(T)):
            tc = np.where(T == TIND)[0]
            meanvalues[TIND, :] = np.mean(points[tc, :], axis=0)
            stdvalues[TIND, :] = np.std(points[tc, :], axis=0)
        
        # Determines fixation clusters by overlapping distributions in velocity
        # and acceleration state space, assuming Gaussian distributions
        fixationcluster = np.argmin(np.sum(meanvalues[:, 1:3], axis=1))
        T[T == fixationcluster] = 100
        fixationcluster2 = np.where(meanvalues[:, 1] < meanvalues[fixationcluster, 1] + 3 * stdvalues[fixationcluster, 1])[0]
        fixationcluster2 = fixationcluster2[fixationcluster2 != fixationcluster]
        for idx in fixationcluster2:
            T[T == idx] = 100
        T[T != 100] = 2
        T[T == 100] = 1

        fixationindexes = np.where(T == 1)[0]
        fixationtimes = BehavioralIndex(fixationindexes)
        fixationtimes = fixationtimes[:, np.where(np.diff(fixationtimes, axis=0) >= 25)[0]+1]

        notfixations = []
        for ii in range(fixationtimes.shape[1]):
            altind = np.arange(fixationtimes[0, ii]-50, fixationtimes[1, ii]+51)
            altind = altind[(altind >= 0) & (altind < len(points))]
            altind = [int(x) for x in altind if x ==x]

            POINTS = points[altind, :]
            sil = np.zeros(5)
            for numclusts in range(1, 6):
                T = KMeans(n_clusters=numclusts, n_init=5).fit(POINTS[::5, :])
                silh = InterVSIntraDist(POINTS[::5, :], T.labels_)
                sil[numclusts-1] = np.mean(silh)
            sil[sil > 0.9 * np.max(sil)] = 1
            numclusters = np.where(sil == np.max(sil))[0]

            # (np.median(numclusters) is NaN so randomly set to 4 ### Fix this
            T = KMeans(n_clusters=int(np.ceil(4)), n_init=5).fit(POINTS)
            print(T.labels_)
            rng = np.zeros((np.max(T.labels_), 2 * (POINTS.shape[1] - 1)))
            print(rng)
            print(rng.shape)
            medianvalues = np.zeros((np.max(T.labels_), POINTS.shape[1]))
            for TIND in range(np.max(T.labels_)):
                tc = np.where(T.labels_ == TIND)[0]
                if len(tc) == 1:
                    rng[TIND, :] = np.ones(2 * (rng.shape[1] // 2))
                    medianvalues[TIND, :] = POINTS[tc, :]
                else:
                    rng[TIND, :] = 5 #
                    medianvalues[TIND, :] = np.median(POINTS[tc, :], axis=0)



        break
    else:
        print("Exit")



[3 3 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 2 2 2 3 3 2 2 2 2 2 2 2 2 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 2 2 2 2 2 2 2 2 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 3 2 2 2 2 2 0 0 0 0 0 2 2 2 2 2
 2 3 2 2 2 2 2 0 0 0 0 0 0 0 0 2 2 0 0 3 3 3 0 0 0 0 0 0 0 0 0 0]
[[0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]]
(3, 6)
[2 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 3 3 3 3 2 2 3 3 3 3 3 3 3 3 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 3 3 3 3 3 3 3 3 3 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 3 3 3